In [2]:
using HomotopyContinuation
using LinearAlgebra

<hr style="border:1px solid gray"> </hr>

# Lecture: Total degree and polyhedral homotopies

<hr style="border:1px solid gray"> </hr>

## Recap

Our task is to solve a system of polynomial equations $F(x) = 0$ given by 

$$ F(x) = F(x_1, \ldots, x_n) = (f_1(x), \ldots, f_n(x)).$$

Recall from Sascha's lecture that, in order to do this, we think of $F$ as a member of a *family* of polynomial systems ${\cal F}_Q$, parametrized by an irrecudible affine variety $Q$. That is, 

$$ F = F_p \in {\cal F}_Q \quad \text{ for some } p \in Q.$$

Recall that we use the notation ${\cal N}(q,Q)$ for the number of regular, isolated solutions of $F_q \in {\cal F}_Q$.

A *generic member* of ${\cal F}_Q$ is a polynomial system with ${\cal N}(Q)$ regular, isolated solutions (you can take this as a definition). By the Parameter Continuation Theorem, there exists a proper subvariety $\Sigma \subsetneq Q$ such that all members $F_q, q \in Q \setminus \Sigma$ are generic. 

If $p, q \in Q \setminus \Sigma$, i.e. ${\cal N}(q,Q) = {\cal N}(p,Q) = {\cal N}(Q)$, and we know all solutions to $F_q(x) = 0$, we can track them to all solutions of $F_p(x) = 0$ along a suitably random path $\gamma$ connecting $q$ and $p$. 

<br>

<img src="PicturesSANNA-1-1.png" width="500"> 

<br>

<hr style="border:1px solid gray"> </hr>

## What is a good choice for ${\cal F}_Q$?

There are infinitely many choices for the family ${\cal F}_Q$. A reasonable choice of ${\cal F}_Q$ should satisfy at least the following two criteria.

* We are able to solve $F_q(x) = 0$ for some $q \in Q \setminus \Sigma$, and *this should be easier than solving $F_p(x) = 0$*.


* We would like $F = F_p \in {\cal F}_Q$ such that ${\cal N}(p,Q) \approx {\cal N}(Q)$. Preferably, we would like $p \in Q \setminus \Sigma$. 

*Caveat: unlike in the offline-online approach, we're assuming here that we only want to solve one particular system of equations $F(x) = 0$. In case we want to solve many systems from the same family ${\cal F}_Q$, it is reasonable to spend more effort on the offline computation.*

Here are some simple examples: 

* Why not choose ${\cal F}_Q = \{ F \}, Q = \{\text{pt} \}$? 


* Consider $F(x) = 2x + 1$. What is ${\cal N}(Q)$ for the families

$$ F(x;b) = b x + 1 \qquad \qquad \qquad \qquad \qquad F(x;a,b) = ax^2 + bx + 1 $$

<br>

<img src="picture3cropped.png" width="700"> 

<br>


In this lecture, we will discuss two important choices for ${\cal F}_Q$: 

* Total degree families ${\cal F}(d_1, \ldots, d_n)$.

\begin{align*}
\text{✅} &\text{Solving $F_q(x) = 0$ is very easy.}\\[0.5em]
\text{❌} &\text{In practice, often ${\cal N}(Q) \gg {\cal N}(p,Q)$}.
\end{align*}

* Sparse families or polyhedral families ${\cal F}({\scr A}_1, \ldots, {\scr A}_n)$.

\begin{align*}
\text{✅} &\text{Often ${\cal N}(Q)$ is much closer to ${\cal N}(p,Q)$ than for the respective total degree family.}\\[0.5em]
\text{❌} &\text{Solving $F_q(x) = 0$ takes a bit more work}.
\end{align*}

To illustrate these claims, try to run the following piece of code and explain what's going on. 

In [ ]:
n = 9
@polyvar x[1:n]
mons_simplex = monomials((1+sum(x))^2)
mons_block = monomials(prod([1+y for y in x]))
F1 = [randn(length(mons_simplex))'*mons_simplex for i = 1:n-4]
F2 = [randn(length(mons_block))'*mons_block for i = n-3:n]
F = [F1;F2]
#solve(F ; start_system = :total_degree)
solve(F ; start_system = :polyhedral) # This is the default.

<hr style="border:1px solid gray"> </hr>

## Total degree homotopies

### Total degree families 

Let $R = \mathbb{C}[x_1,\ldots, x_n]$ and $R_{\leq d} = \bigoplus_{|\alpha| \leq d} \mathbb{C} \cdot x^\alpha$. For an $n$-tuple $(d_1, \ldots, d_n) \in (\mathbb{N} \setminus \{0\})^n$ we define the *total degree family*

\begin{align*}
{\cal F}(d_1,\ldots, d_n) &:= \{ F = (f_1, \ldots, f_n) ~|~ f_i \in R_{\leq d_i} \} \\
&= R_{\leq d_1} \times \cdots \times R_{\leq d_n} \\ 
&\simeq Q := \mathbb{C}^{D_1} \times \cdots \times \mathbb{C}^{D_n},
\end{align*}

where $D_i = \text{dim}_{\mathbb{C}} R_{\leq d_i}$ and we can think of the standard coordinates on $Q$ as the coefficients of $f_1, \ldots, f_n$. We recall the following theorem, which we saw before in Paul's lecture. 

**Theorem**</u> (Bézout's theorem):
<p style="border:3px; border-style:solid; padding: 0.2em; text-align:center;"> 
For any $F \in {\cal F}(d_1, \ldots, d_n)$, the number of isolated solutions to $F(x) = 0$ is bounded by 
$$ {\cal N}_{\text{Béz}} := \prod_{i=1}^n d_i.$$
Moreover, generic members of ${\cal F}(d_1, \ldots, d_n)$ have exactly ${\cal N}_{\text{Béz}}$ regular, isolated solutions. That is, ${\cal N}(Q) = {\cal N}_{\text{Béz}}$.
</p>

As an example, consider the case where $n = 2$ and $(d_1, d_2) = (2,2)$. 

$$ {\cal F}(2,2) = \begin{pmatrix}
f_1 = a_{00} + a_{10} x + a_{01} y + a_{20}x^2 + a_{11}xy + a_{02} y^2 \\
f_2 = b_{00} + b_{10} x + b_{01} y + b_{20}x^2 + b_{11}xy + b_{02} y^2 \\
\end{pmatrix} $$
<img src="Pictures SANNA-4-1.png" width="300"> 


We have ${\cal N}_{\text{Béz}} = 4$. There are strictly less than 4 solutions if either

* 2 or more solutions *coincide*: 
$$ f_1 = f_2 = \det \begin{pmatrix} f_{1x} & f_{1y} \\ f_{2x} & f_{2y} \end{pmatrix} = 0 \quad \text{has a solution},$$
which is equivalent to the vanishing of a nonzero polynomial in the coefficients of $f_1, f_2$ called the *discriminant*. 


* or there are solutions *at infinity*:
$$ \det A_\infty = \det \begin{pmatrix} a_{20} & a_{11} & a_{02} & \\ & a_{20} & a_{11} & a_{02} \\ b_{20} & b_{11} & b_{02} & \\ & b_{20} & b_{11} & b_{02} \end{pmatrix} = 0.$$

**Question**: what about the case where $f_1 = f_2 = 0$ has infinitely many solutions?

Here is some code to illustrate this. Let us first solve a random member. 

In [7]:
@var x y a[0:2,0:2] b[0:2,0:2]
F = [sum([a[i,j]*x^(i-1)*y^(j-1) for i=1:3 for j = 1:4-i]); sum([b[i,j]*x^(i-1)*y^(j-1) for i=1:3 for j = 1:4-i])]
F = System(F, parameters = setdiff(variables(F),[x;y]))
solve(F;target_parameters = randn(12))

Result with 4 solutions
• 4 paths tracked
• 4 non-singular solutions (2 real)
• random_seed: 0x51c26461
• start_system: :polyhedral


Next, we construct a member with a solution at infinity, by imposing the vanishing of $\det A_\infty$.

In [8]:
a2 = [a[i,4-i] for i = 3:-1:1]
b2 = [b[i,4-i] for i = 3:-1:1]
A = [[a2' 0]; [0 a2']; [b2' 0]; [0 b2']]

4×4 Array{Expression,2}:
 a₂₋₀  a₁₋₁  a₀₋₂     0
    0  a₂₋₀  a₁₋₁  a₀₋₂
 b₂₋₀  b₁₋₁  b₀₋₂     0
    0  b₂₋₀  b₁₋₁  b₀₋₂

In [9]:
parsys = System([randn(11,12)*parameters(F) + randn(11); det(A)], variables = parameters(F))
R = solve(parsys)
solve(F;target_parameters = solutions(R)[1])

Result with 3 solutions
• 4 paths tracked
• 3 non-singular solutions (0 real)
• random_seed: 0x225b36c8
• start_system: :polyhedral


### Total degree start systems

For $(d_1, \ldots, d_n)$, we consider the associated total degree family ${\cal F}(d_1, \ldots, d_n)$ and define $q \in Q$ such that 

$$ F_q(x) = (x_1^{d_1} -1,~ x_2^{d_2}-1, \ldots,~ x_n^{d_n}-1 ) \quad \in {\cal F}(d_1, \ldots, d_n).$$

The set of solutions to $F_q(x) = 0$ is 
$$ W_{d_1} \times W_{d_2} \times \cdots \times W_{d_n} \subset \mathbb{C}^n, \quad \text{where } W_{d_i} = \{ d_i \text{-th roots of 1} \}.$$

Note that ${\cal N}(q,Q) = {\cal N}(Q) = {\cal N}_{\text{Béz}} = \prod_{i=1}^n d_i$, so that $q \notin \Sigma$. 

In Sascha's lecture, we saw that for generic $q^* \in Q \setminus \Sigma$ the *straight line homotopy* $H(x,t) = t F_{q^*}(x) + (1-t)F_p(x)$ has smooth solution paths whose endpoints, for $t$ from 1 to 0, include all isolated zeros of $F_p$. In practice, the parameters $q$ above are not generic in this sense. It turns out that in order to "randomize" our $q$, it suffices to use $\lambda q$ for some random complex $\lambda \neq 0$ (note that this does not change the solutions: $F_q(x) = 0 \Leftrightarrow F_{\lambda q}(x) = 0$). 

$$ H(x,t) = t F_{q}(x) + (1-t)F_p(x) \qquad \qquad \qquad \qquad H(x,t) = \lambda t F_{q}(x) + (1-t)F_p(x).$$

<br>

<img src="picture5cropped.png" width="700"> 

<br>



### Algorithm 

This discussion leads to the following algorithm for computing the isolated solutions of $F(x) = 0$. 

1. For $i = 1, \ldots, n$, set $d_i = \min \{ d \in \mathbb{N} ~|~ f_i \in R_{\leq d} \}$.

2. Set $F_q = (x_1^{d_1} -1,~ x_2^{d_2}-1, \ldots,~ x_n^{d_n}-1 )$.

3. Track the ${\cal N}_{\text{Béz}}$-many solutions $ W_{d_1} \times W_{d_2} \times \cdots \times W_{d_n}$ along the homotopy $H(x,t) = \lambda t F_{q}(x) + (1-t)F_p(x)$ for $t$ going from 1 to 0. 

<hr style="border:1px solid gray"> </hr>

## Polyhedral homotopies

### Introduction: improving Bézout's bound

Consider the subfamily 

$$ {\cal F}_Q = \begin{pmatrix}
f_1 = a_{00} + a_{10}x + a_{10}y + a_{11}xy \\
f_2 = b_{00} + b_{10}x + b_{10}y + b_{11}xy \\
\end{pmatrix} \subset {\cal F}(2,2)$$

**Question**: what is ${\cal N}(Q)$ for this family?

**Hint**: For a general member $F_q \in {\cal F}_Q$, we can find an invertible $2 \times 2$ matrix $T$ such that 

$$ T \begin{pmatrix} f_1 \\ f_2 \end{pmatrix} = \begin{pmatrix}
c_{00} + c_{10}x + c_{10}y + c_{11}xy \\
d_{00} + d_{10}x + d_{10}y \\
\end{pmatrix}.$$

**Question**: what does this tell you about the degree of the surface in $\mathbb{P}^3$ obtained by taking the closure of the image of $(x,y) \mapsto (1:x:y:xy)$?

The reason for the discrepancy ${\cal N}(Q) < {\cal N}_{\text{Béz}}$ is that **a generic member of ${\cal F}_Q$ is not a generic member of ${\cal F}(2,2)$**:

$$ \begin{matrix}
f_1 = a_{00} + a_{10}x + a_{10}y + a_{11}xy + 0 \cdot x^2 + 0 \cdot y^2 \\
f_2 = b_{00} + b_{10}x + b_{10}y + b_{11}xy + 0 \cdot x^2 + 0 \cdot y^2 \\
\end{matrix} $$

In fact, you can easily check that for all members of ${\cal F}_Q \subset {\cal F}(2,2)$, $\det A_\infty = 0$. 

This suggests the following question.

<p style="border:3px; border-style:solid; padding: 0.2em; text-align:center;"> 
Can we improve Bézout's bound if we know which monomials occur in the $f_i$?
</p>

It turns out we can, and HomotopyContinuation.jl seems to know about this.

In [13]:
@polyvar x y 
F = [randn(4)'*monomials((1+x)*(1+y)); randn(4)'*monomials((1+x)*(1+y))]
solve(F)

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  2
  # non-singular solutions (real):  2 (2)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (2)


Result with 2 solutions
• 2 paths tracked
• 2 non-singular solutions (2 real)
• random_seed: 0xf0da2d71
• start_system: :polyhedral


### Newton polytopes

For a polynomial $f = \sum_{\alpha} c_\alpha x^\alpha \in R = \mathbb{C}[x_1,\ldots,x_n]$, we define 

* The **support** of $f$: $$\text{Supp}(f) = \{\alpha \in \mathbb{N}^n ~|~ c_\alpha \neq 0 \} \subset \mathbb{N}^n.$$
* The **Newton polytope** of $f$: $$\text{Newt}(f) = \text{Conv}(\text{Supp}(f)) \subset \mathbb{R}^n.$$

Example: $$f = a_{00} + a_{11} xy + a_{21}x^2y + a_{12}xy^2, \qquad \qquad f = (1+x)(1+y)(1+z).$$

<br>

<img src="polyh1-1.png" width="600"> 

<br>

The set of convex polytopes in $\mathbb{R}^n$ admits a commutative and associative binary operation $+$ called **Minkowski addition**. The Minkowski sum of two convex polytopes $P_1, P_2 \subset \mathbb{R}^n$ is 

$$ P_1 + P_2 = \{ \alpha_1 + \alpha_2 ~|~ \alpha_1 \in P_1, \alpha_2 \in P_2 \}.$$

Here $\alpha_1 + \alpha_2$ is a usual sum of vectors in $\mathbb{R}^n$.

**Exercise**: Show that $+$ is commutative, associative and that the set of convex polytopes is closed under Minkowksi addition. 

For a positive real number $\lambda$, the $\lambda$**-dilation** of a convex polytope $P \subset \mathbb{R}^n$ is 

$$ \lambda \cdot P = \{ \lambda \alpha ~|~ \alpha \in P \}.$$

<br>

<img src="polyh1-2.png" width="500"> 

<br>

Given $n$ convex polytopes $P_1, \ldots, P_n \subset \mathbb{R}^n$, the Euclidean volume

$$ {\cal P}(\lambda_1, \ldots, \lambda_n) = \text{Vol}_n(\lambda_1 \cdot P_1 + \cdots + \lambda_n \cdot P_n)$$

is a homogeneous polynomial of degree $n$. The **Mixed volume** $ \text{MV}(P_1, \ldots, P_n)$ of $P_1, \ldots, P_n$ is the coefficient standing with $\lambda_1\lambda_2 \cdots \lambda_n$ in $\mathcal{P}(\lambda_1, \ldots, \lambda_n)$.

A useful formula in the case $n = 2$ is $\text{MV}(P_1,P_2) = \text{Vol}_2(P_1 + P_2) - \text{Vol}_2(P_1) - \text{Vol}_2(P_2)$.

<br>

<img src="polyh1-3.png" width="500"> 

<br>

**Exercise**: Verify this formula for $P_1 = P_2 = [0,1]^2 \subset \mathbb{R}^2$. More generally, compute $\text{MV}(P_1, \ldots, P_n)$ for $P_i = [0,1]^n \subset \mathbb{R}^n$ for all $i$.

### Sparse families

For a finite subset ${\scr A}_i \subset \mathbb{N}^n$, let $R_{{\scr A}_i} = \bigoplus_{\alpha \in {\scr{A}_i}} \mathbb{C} \cdot x^\alpha$. For an $n$-tuple of supports ${\scr A} = ({\scr A}_1, \ldots, {\scr A}_n)$ we define the *sparse family* 

\begin{align*}
{\cal F}({\scr A}) &:= \{ F = (f_1, \ldots, f_n) ~|~ f_i \in R_{{\scr A}_i} \} \\
&= R_{{\scr A}_1} \times \cdots \times R_{{\scr A}_n} \\ 
&\simeq Q := \mathbb{C}^{D_1} \times \cdots \times \mathbb{C}^{D_n},
\end{align*}

where $D_i = \dim_{\mathbb{C}} R_{{\scr A}_i} = | {\scr A}_i |$ and we can think of the standard coordinates on $Q$ as the coefficients of $f_1, \ldots, f_n$. In what follows, we will write 

$$ P_i = \text{Conv}({\scr A}_i), \qquad i = 1, \ldots, n.$$

**Theorem**</u> (Bernstein, Khovanskii, Kushnirenko (BKK) theorem):
<p style="border:3px; border-style:solid; padding: 0.2em; text-align:center;"> 
For any $F \in {\cal F}({\scr A})$, the number of isolated solutions to $F(x) = 0$ in $(\mathbb{C} \setminus \{0\})^n$ is bounded by 
\begin{align*}
    &\\
    & {\cal N}_{\text{BKK}} := \text{MV}(P_1, \ldots, P_n).\\
    &
    \end{align*}
Moreover, generic members of ${\cal F}({\scr A})$ have exactly ${\cal N}_{\text{BKK}}$ regular, isolated solutions in $(\mathbb{C} \setminus \{0\})^n$. 
</p>

In the case where $0 \in {\scr A}_i$ for $i = 1, \ldots, n$, we can replace $(\mathbb{C} \setminus \{0\})^n$ in the above theorem by $\mathbb{C}^n$. In that case, the theorem implies ${\cal N}(Q) = {\cal N}_{\text{BKK}}$. **I will make this assumption from now on.**

**Remark**: Note that ${\cal N}_{\text{BKK}}$ only depends on the convex hull of the supports ${\scr A}_i$. For this reason, we also refer to the families ${\cal F}(A)$ as **polyhedral families**, and similarly for the associated homotopies. 

**Example**: Let $\Delta_n$ be the $n$-dimensional standard simplex. If ${{\scr A}_i} = (d_i \cdot \Delta_n) \cap \mathbb{N}^n$, then ${\cal F}({\scr A}) = {\cal F}(d_1, \ldots, d_n)$ and ${\cal N}_{\text{BKK}} = {\cal N}_{\text{Béz}}$.

**Example**: Consider again the family 

$$\begin{pmatrix}
a_{00} + a_{10}x + a_{10}y + a_{11}xy \\
b_{00} + b_{10}x + b_{10}y + b_{11}xy \\
\end{pmatrix} \subset {\cal F}(2,2).$$

This is the sparse/polyhedral family ${\cal F}({\scr A})$ where ${\scr A}_1 = {\scr A}_2 = \{(0,0),(1,0),(0,1),(1,1) \}$. We have checked that ${\cal N}_{\text{BKK}} = 2$.

**Example**: Consider the family 

$$\begin{pmatrix}
a_1 +  a_2 xy + a_3 x^2y + a_4 xy^2 \\
b_1 + b_2 x + b_3 y + b_4 xy \\
\end{pmatrix} \subset {\cal F}(3,2).$$

This is the sparse/polyhedral family ${\cal F}({\scr A})$ where ${\scr A}_1 = \{(0,0),(1,1),(2,1),(1,2) \}, {\scr A}_2 = \{(0,0),(1,0),(0,1),(1,1) \}$. We have seen the polygons $P_1, P_2$ before. Their mixed volume is ${\cal N}_{\text{BKK}} = 4$. Note that the smallest total degree family containing ${\cal F}({\scr A})$ is ${\cal F}(3,2)$, with ${\cal N}_{\text{Béz}} = 6$.

In [7]:
@var x y a[1:4] b[1:4]
F = [dot(a,[1,x*y,x^2*y,x*y^2]),dot(b,[1,x,y,x*y])]
F = System(F, parameters = [a;b])

System of length 2
 2 variables: x, y
 8 parameters: a₁, a₂, a₃, a₄, b₁, b₂, b₃, b₄

 a₁ + x*y*a₂ + x*y^2*a₄ + x^2*y*a₃
 b₁ + x*b₂ + y*b₃ + x*y*b₄

In [8]:
solve(F;target_parameters = randn(8))

Result with 4 solutions
• 4 paths tracked
• 4 non-singular solutions (2 real)
• random_seed: 0xebc5c8d2
• start_system: :polyhedral


**Example** (${\cal N}_{\text{BKK}} \ll {\cal N}_{\text{Béz}}$): This is an example taken from [Huber-Sturmfels]. Consider the family 

$$\begin{pmatrix}
a_1 +  a_2 x + a_3 x^ky^k \\
b_1 + b_2 y + b_3 x^ky^k 
\end{pmatrix} \subset {\cal F}(2k,2k).$$

Show that $\lim_{k \rightarrow \infty} ( {\cal N}_{\text{BKK}} / {\cal N}_{\text{Béz}} ) = 0$.

### Sparse start systems 

In order to solve our system $F(x) = F_p(x) \in {\cal F}({\scr A})$, we need to solve a start system $F_q(x) = 0$ where $q$ is generic. We write

$$ F_q(x) = (g_1(x), ~ g_2(x), ~ \cdots , ~ g_n(x) ), \quad \text{where } \quad g_i(x) = \sum_{\alpha \in {\scr A}_i} c_{i,\alpha} x^\alpha$$

for random coefficients $c_{i,\alpha} \in \mathbb{C}$. The first step is to introduce a new variable $t$, to obtain 

$$ \hat{F}_q(x,t) = (\hat{g}_1(x,t), ~ \hat{g}_2(x,t), ~ \cdots , ~ \hat{g}_n(x,t) ), \quad \text{where } \quad \hat{g}_i(x,t) = \sum_{\alpha \in {\scr A}_i} c_{i,\alpha} x^\alpha t^{w_{i,\alpha}}$$

for some integers $w_{i,\alpha}, {i = 1, \ldots, n, ~ \alpha \in {\scr A}_i}$. Note that $F_q(x) = \hat{F}_q(x,1)$.

The solutions to the *lifted* system $\hat{F}_q(x,t) = 0$ are given by algebraic functions 

$$x(t) = (x_1(t), \ldots, x_n(t)) \quad \text{such that} \quad \hat{F}_q(x(t),t) = 0.$$

These algebraic functions have ${\cal N}_{\text{BKK}}$ branches. Around $t = 0$, such a solution branch $x(t)$ looks like 

$$ x(t) = (X_1 t^{e_1}, ~ X_2 t^{e_2}, \ldots, ~ X_n t^{e_n}) ~ + \text{ higher order terms},  $$

For some $X = (X_1, \ldots, X_n) \in (\mathbb{C} \setminus \{0\})^n$ and $e = (e_1, \ldots, e_n) \in \mathbb{Q}^n$.

<p style="border:3px; border-style:solid; padding: 0.2em; text-align:center;"> 
Compute $X, e$ to approximate the branches of $x(t)$ around $t = 0$ and continue for $t$ from $0$ to $1$.
</p>

* For sufficiently generic $w_{i,\alpha}$, the exponents $e$ can be obtained by enumerating mixed cells in a coherent fine mixed subdivision of $P_1 + \cdots + P_n$.
* The corresponding leading coefficients $X$ are the solutions to a system of binomial equations, which is *easy* to solve.

### Algorithm

This discussion leads to the following algorithm for computing the isolated solutions of $F(x) = 0$. 

1. For $i = 1, \ldots, n$, set ${\scr A}_i = \text{Supp}(f_i) \cup \{0\}$.

2. Solve a random member $F_q(x) \in {\cal F}({\scr A})$ via the lifting procedure.

3. Track the ${\cal N}_{\text{BKK}}$-many solutions of $F_q(x) = 0$ along the homotopy $H(x,t) = t F_{q}(x) + (1-t)F(x)$ for $t$ going from 1 to 0. 


**Question**: why don't we need $\lambda$ here?
<hr style="border:1px solid gray"> </hr>

## Conclusions
